In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [6]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [8]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)

(66, 1023)


In [9]:
filtered_joined_df = prep.filter_outliers_IQR(joined_df, "survival_time", coef=1.5)

# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(filtered_joined_df, rand)

## Get baselines

In [11]:
mean_baseline = mean_squared_error(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
median_baseline = mean_absolute_error(y_df.values, np.repeat(np.median(y_df.values.squeeze()), y_df.shape[0]))
r2_baseline = r2_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
expl_var_baseline = explained_variance_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))

print(f"L2 baseline: {mean_baseline}")
print(f"L1 baseline: {median_baseline}")
print(f"R2 baseline: {r2_baseline}")
print(f"explained variance baseline: {expl_var_baseline}")

L2 baseline: 216346.93791991402
L1 baseline: 353.1803278688525
R2 baseline: 0.0
explained variance baseline: 0.0


In [12]:
def objective(h_params, X, y, loss_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingRegressor(
        # We use lad since it most closely matches up with hyper-parameter objective
        loss=loss_default,
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [13]:
space = [
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * (len(space))
n_calls = 50 * (len(space))

In [14]:
loss_default = "ls"
scoring_default = "explained_variance"
callback_file = f"{unified_dsets[dset_idx]}_opt_gbr_h_params_{scoring_default}_outliers_removed.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.06277134222741902, 204, 3, 'log2', 5, 2]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.8045
Function value obtained: -0.1724
Current minimum: -0.1724
Iteration No: 2 started. Evaluating function at random point.
[0.048820392162007745, 987, 4, 'log2', 2, 3]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.8086
Function value obtained: -0.1951
Current minimum: -0.1951
Iteration No: 3 started. Evaluating function at random point.
[0.06070295271268183, 591, 3, 'log2', 4, 2]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.6172
Function value obtained: -0.1702
Current minimum: -0.1951
Iteration No: 4 started. Evaluating function at random point.
[0.08767022733772234, 559, 4, 'log2', 4, 2]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.5651
Function value obtained: -0.1920
Current minimum: -0.1951
Iteration No: 5 started. Evaluating functio

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 76 ended. Search finished for the next optimal point.
Time taken: 1.4588
Function value obtained: -0.2622
Current minimum: -0.2840
Iteration No: 77 started. Searching for the next optimal point.
[0.1, 500, 2, 'auto', 6, 1]
Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 3.4344
Function value obtained: -0.2713
Current minimum: -0.2840
Iteration No: 78 started. Searching for the next optimal point.
[0.068170997375156, 100, 2, 'auto', 6, 1]
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 1.5040
Function value obtained: -0.2577
Current minimum: -0.2840
Iteration No: 79 started. Searching for the next optimal point.
[0.1, 1000, 2, 'auto', 6, 1]
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 5.7702
Function value obtained: -0.2737
Current minimum: -0.2840
Iteration No: 80 started. Searching for the next optimal point.
[0.041433669501713136, 859, 2, 'auto', 6, 1]
Iteration No: 80 end

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 2.9427
Function value obtained: -0.2558
Current minimum: -0.3089
Iteration No: 101 started. Searching for the next optimal point.
[0.1, 100, 2, 'auto', 6, 3]


...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 11.9120
Function value obtained: -0.2901
Current minimum: -0.3178
Iteration No: 299 started. Searching for the next optimal point.
[0.1, 100, 2, 'auto', 6, 2]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 11.7636
Function value obtained: -0.2628
Current minimum: -0.3178
Iteration No: 300 started. Searching for the next optimal point.
[0.1, 100, 2, 'auto', 6, 1]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 11.2113
Function value obtained: -0.2879
Current minimum: -0.3178
